In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Import Neighborhood data of Amsterdam, clean up the data

In [2]:
ams = pd.read_csv('Amsterdam_Neighborhood_data.csv')

# select columns and drop first two rows
filterColum = ['subject', 'regio_type', 'ninhabitants', 'lon', 'lat']            
ams = ams[filterColum].iloc[2:,:]

# rename the columns and reset the index
ams = ams.rename(columns={"subject": "Neighborhood", "lon": "Longitude", "lat": "Latitude"})
ams = ams.reset_index(drop=True)

# convert all str type columns into float
ams['ninhabitants'] = pd.to_numeric(ams['ninhabitants'], errors='coerce')
ams['Longitude'] = pd.to_numeric(ams['Longitude'], errors='coerce')
ams['Latitude'] = pd.to_numeric(ams['Latitude'], errors='coerce')
ams.head()

C:\Users\YI\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254

,Neighborhood,regio_type,ninhabitants,Longitude,Latitude
0,Burgwallen-Oude Zijde,Wijk,4280.0,4.896943,52.372566
1,Kop Zeedijk,Buurt,1020.0,4.900280,52.375937
2,Oude Kerk e.o.,Buurt,670.0,4.896156,52.373793
3,Burgwallen Oost,Buurt,1610.0,4.898757,52.372815
4,Nes e.o.,Buurt,370.0,4.893728,52.370861


## 2. Separate the dataframe by two region types: Buurt vs. Wijk 

In [3]:
# drop the neighborhood with too less population
amsTop = ams.loc[ams['ninhabitants'] >= 3000]
amsTop = amsTop.reset_index(drop=True)
print(len(ams))
print(len(amsTop))

# separatly save dfs of Buurt and Wijk
amsBuurt = amsTop.loc[amsTop['regio_type'] == 'Buurt']
amsBuurt = amsBuurt.reset_index(drop=True)
amsWijk = amsTop.loc[amsTop['regio_type'] == 'Wijk']
amsWijk = amsWijk.reset_index(drop=True)

print(len(amsBuurt), len(amsWijk))

581
176
97 79


## 3. Visualize two type of regions on Amsterdam map

In [4]:
# define the map coordinate of Amsterdam
address = 'Amsterdam'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Amsterdam are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Amsterdam are 52.3727598, 4.8936041.


In [5]:
# adding two sets of neighborhoods onto map

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=12)

neighborhoods = amsBuurt
# add markers to map
for lat, lng, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
neighborhoods = amsWijk
# add markers to map
for lat, lng, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork) 
    
map_newyork

In [6]:
CLIENT_ID = 'GITTZBK0YLFZKHX4UVFZSEEJFK4NVAP2USZ3VF4RGW11OX3G' # your Foursquare ID
CLIENT_SECRET = '1R34U3IXM3Q5AUJVG2AAZJ4CREO2UNPBSJPNY5DMBDQFHXB1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

print(amsBuurt.loc[amsBuurt['Neighborhood'] == 'Cremerbuurt West'])
lat = amsTop.loc[3, 'Latitude'] 
lon = amsTop.loc[3, 'Longitude'] 
lat = amsBuurt.loc[3, 'Latitude'] 
lon = amsBuurt.loc[3, 'Longitude'] 


neighborhood_latitude = lat   # neighborhood latitude value
neighborhood_longitude = lon   # neighborhood longitude value

neighborhood_name = 'Amsterdam' # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))
print(lat)
print(lon)
amsTop.head()

Your credentails:
CLIENT_ID: GITTZBK0YLFZKHX4UVFZSEEJFK4NVAP2USZ3VF4RGW11OX3G
CLIENT_SECRET:1R34U3IXM3Q5AUJVG2AAZJ4CREO2UNPBSJPNY5DMBDQFHXB1
       Neighborhood regio_type  ninhabitants  Longitude   Latitude
9  Cremerbuurt West      Buurt        5620.0   4.860385  52.360492
Latitude and longitude values of Amsterdam are 52.38360632, 4.87390094.
52.38360632
4.87390094


,Neighborhood,regio_type,ninhabitants,Longitude,Latitude
0,Burgwallen-Oude Zijde,Wijk,4280.0,4.896943,52.372566
1,Burgwallen-Nieuwe Zijde,Wijk,3960.0,4.895518,52.375703
2,Grachtengordel-West,Wijk,6475.0,4.886836,52.373709
3,Grachtengordel-Zuid,Wijk,5440.0,4.893349,52.364821
4,Nieuwmarkt/Lastage,Wijk,9605.0,4.905641,52.372436


In [7]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5eb1600daba297001cbad5aa'},
 'response': {}}

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: 'groups'

## Get the top 100 venues within a radius of 500 meters of each Neighborhood.

In [9]:
# make use of foursquare API to get the venues, given the coordinate of a neighborhood.
# parameters setup
CLIENT_ID = 'GITTZBK0YLFZKHX4UVFZSEEJFK4NVAP2USZ3VF4RGW11OX3G'      # your Foursquare ID
CLIENT_SECRET = '1R34U3IXM3Q5AUJVG2AAZJ4CREO2UNPBSJPNY5DMBDQFHXB1'  # your Foursquare Secret
VERSION = '20180605'              # Foursquare API version
LIMIT = 100                       # limit of number of venues returned by Foursquare API
radius = 500                      # define radius

# the function goes through each neigborhoods of the input df by a loop
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    cnt = 0
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        cnt+=1
    print('num of Neighborhood: ',cnt)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print('Done')
    return(nearby_venues)

In [10]:
# venues of all the Amsterdam neighborhoods
amsTop_venues = getNearbyVenues(names=amsTop['Neighborhood'],
                                   latitudes=amsTop['Latitude'],
                                   longitudes=amsTop['Longitude']
                                  )
amsTop_venues.iloc[0:3,:]

KeyError: 'groups'

## Explore the venue dataset

## One-hot-key encoding + groupby() for calcualting the frequency table

In [ ]:
def convert_2_oneHotkey_Grouped_pd(df_venue):
    # one-hot key
    df_onehot = pd.get_dummies(df_venue[['Venue Category']], prefix="", prefix_sep="")

    # add neighborhood column back to dataframe
    df_onehot['Neighborhood'] = df_venue['Neighborhood'] 

    # move neighborhood column to the first column
    fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
    df_onehot = df_onehot[fixed_columns]

    # groupby to get to merge rows and calculate freq 
    df_group = df_onehot.groupby('Neighborhood')
    df_group = df_group.mean().reset_index()

    print(df_group.shape)
    return df_group

amsTop_group = convert_2_oneHotkey_Grouped_pd(amsTop_venues)

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_group['Neighborhood']

for ind in np.arange(Toronto_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Cluster Neighborhoods of Amsterdam